In [1]:
import xml.etree.cElementTree as ET
from IPython.display import clear_output
from ipywidgets import FloatProgress, IntText
from IPython.display import display
from nltk.tokenize.stanford import StanfordTokenizer
import time

In [7]:
p = ET.iterparse("data/enwiki-20170820-pages-articles.xml", 
                 events=("start", "end"))

In [ ]:
start = time.time()
article_count = 0
root = None
f = FloatProgress(min=0, max=17773690)
t = IntText(value=0, description="Articles")
display(t, f)

for event, elem in p:
    if root == None:
        root = elem
    if event == "end" and elem.tag == '{http://www.mediawiki.org/xml/export-0.10/}title':
        article_count += 1
        if article_count % 10000 == 0:
            elapsed_time = time.time() - start
            f.value = article_count
            t.value = article_count
            
    root.clear()
    
print("Articles:", article_count, "Time:", (time.time() - start), "seconds")

In [ ]:
article_count

In [6]:
start = time.time()
article_count = 0
root = None
f = FloatProgress(min=0, max=17773690)
t = IntText(value=0, description="Articles")
m = IntText(value=0, description="Matching Articles")
display(t, m, f)

fh = open("hello.txt","w")

for event, elem in p:
    if root == None:
        root = elem
    if event == "end" and elem.tag == '{http://www.mediawiki.org/xml/export-0.10/}title':
        article_count += 1
        if article_count % 10000 == 0:
            elapsed_time = time.time() - start
            f.value = article_count
            t.value = article_count
        if "hemist" in elem.text:
            m.value += 1 
            fh.write(elem.text)
            
    root.clear()
    
fh.close()
print("Articles:", article_count, "Time:", (time.time() - start), "seconds")

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

Articles: 17773690 Time: 1657.1814320087433 seconds


In [8]:
start = time.time()
article_count = 0
root = None
f = FloatProgress(min=0, max=17773690)
t = IntText(value=0, description="Articles")
m = IntText(value=0, description="Matching Articles")
display(t, m, f)

fh = open("scientists.txt","w")

for event, elem in p:
    if root == None:
        root = elem
    if event == "end" and elem.tag == '{http://www.mediawiki.org/xml/export-0.10/}text':
        article_count += 1
        if article_count % 10000 == 0:
            elapsed_time = time.time() - start
            f.value = article_count
            t.value = article_count
        if "Category:Scientist" in elem.text:
            m.value += 1 
            fh.write(elem.text)
            
    root.clear()
    
fh.close()
print("Articles:", article_count, "Time:", (time.time() - start), "seconds")

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

TypeError: argument of type 'NoneType' is not iterable